In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
model = tf.keras.models.load_model('my_model.h5',compile = True)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 10)           200000    
                                                                 
 global_average_pooling1d (G  (None, 10)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 20)                220       
                                                                 
 dense_1 (Dense)             (None, 40)                840       
                                                                 
 dense_2 (Dense)             (None, 3)                 123       
                                                                 
Total params: 201,183
Trainable params: 201,183
Non-trainable params: 0
__________________________________________________

In [4]:
vocab_size = 20000
embedding_dim = 10
max_length = 150
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

In [5]:
data_predict = pd.read_excel("data clear alif.xlsx")
data_predict = data_predict.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
print(f"total null {data_predict.isnull().sum()}")
print(f"total null {data_predict.isna().sum()}")

total null Unnamed: 0    0
text          0
count         0
dtype: int64
total null Unnamed: 0    0
text          0
count         0
dtype: int64


In [6]:
data_predict = data_predict.dropna()
data_predict.isna().sum()
print(f"total null after drop {data_predict.isnull().sum()}")
print(f"total null after drop {data_predict.isna().sum()}")


total null after drop Unnamed: 0    0
text          0
count         0
dtype: int64
total null after drop Unnamed: 0    0
text          0
count         0
dtype: int64


In [7]:
data = data_predict['text'].tolist()
data[:5]

['ayo jawab siapa yg baru tau kalau pos indonesia punya ewallet bernama  pospayofficial  utk mendukung langkah bumn utk negeri install dan aktivasi pospay dr skrg yuk banyak bonus yg akan didapat dan jg bentuk dr aksi nyata kita dukung karya hebat bumn',
 'oktober  memperingati hari kesaktian pancasila bersama  pospayofficial keren sih tagline nya selalu tangguh di mana saja sbg informasi pospay adalah ewallet keluaran bumn pos indonesia',
 'alasan sederhana mau coba download  pospayofficial adl dalam rangka mendukung karya bumn aplikasi ini layak diapresiasi sbg salah satu ikhtiar pos indonesia utk negeri',
 'tercatat sampai dengan semester i pertumbuhan bisnis pt pos indonesia mencapai  dibandingkan dengan tahun lalu semua ini berkat kehadiran platform digital pakaipospay  tiorichardo  airinairinnz  andreopasumual  warhasnotendyet  kontannews',
 'digitalisasi di tubuh pt pos indonesia diwujudkan dengan diluncurkannya dua aplikasi produk layanan untuk semakin memudahkan pelanggan sala

In [8]:
X = data
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(data)
padded = pad_sequences(sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)
predic = model.predict(padded)


In [9]:
y_label = {0:"negative",1:"neutral",2:"positive"}
sentimen = []
for x in predic:
    sentimen.append(y_label[np.argmax(x)])


In [10]:
data_clean = pd.DataFrame({'tweet':data_predict['text'],'sentimen':sentimen,'count':data_predict['count']})
data_clean

,tweet,sentimen,count
0,ayo jawab siapa yg baru tau kalau pos indonesi...,positive,121
1,oktober memperingati hari kesaktian pancasila...,positive,115
2,alasan sederhana mau coba download pospayoffi...,positive,99
3,tercatat sampai dengan semester i pertumbuhan ...,negative,82
4,digitalisasi di tubuh pt pos indonesia diwujud...,neutral,78
...,...,...,...
878,unfortunately pos malaysia only delivers to th...,positive,1
879,kapankah anda terakhir kali menulis surat masi...,negative,1
880,po case album aespa savage ditunggu sampai ...,neutral,1
881,checkout all nominal shopee split pay shopeepa...,positive,1
